# Using deep features to build an image classifier

## Fire up turicreate

In [1]:
import turicreate

# Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [2]:
image_train = turicreate.SFrame('./image_train_data')
image_test = turicreate.SFrame('./image_test_data')

In [5]:
image_train

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.24287176132202148,1.0954537391662598, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.5250879526138306, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.5660159587860107, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.129795789718628, 0.0,0.0, 0.7781944870948792, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.7178692817687988, 0.0,0.0, 0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.5781855583190918, 0.0,0.0, 0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.22067785263061523, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0,0.23753464221954346, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57378625869751, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.6589357256889343, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


In [6]:
image_test

id,image,label,deep_features,image_array
0,Height: 32 Width: 32,cat,"[1.1346900463104248, 0.0,0.0, 0.0, ...","[158.0, 112.0, 49.0,159.0, 111.0, 47.0, ..."
6,Height: 32 Width: 32,automobile,"[0.2313588261604309, 0.0,0.0, 0.0, 0.0, ...","[160.0, 37.0, 13.0,185.0, 49.0, 11.0, 20 ..."
8,Height: 32 Width: 32,cat,"[0.0, 0.0,0.034419238567352295, ...","[23.0, 19.0, 23.0, 19.0,21.0, 28.0, 21.0, 16.0, ..."
9,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 11.6065092086792, ...","[217.0, 215.0, 209.0,210.0, 208.0, 202.0, ..."
12,Height: 32 Width: 32,dog,"[0.3223174810409546, 0.0,1.2493335008621216, 0.0, ...","[91.0, 64.0, 30.0, 82.0,58.0, 30.0, 87.0, 73.0, ..."
16,Height: 32 Width: 32,dog,"[0.0, 0.0,0.34735703468322754, ...","[95.0, 76.0, 78.0, 92.0,77.0, 78.0, 89.0, 77.0, ..."
24,Height: 32 Width: 32,dog,"[1.3155765533447266, 0.0,0.0, 0.0, 0.0, 0.0, ...","[136.0, 134.0, 118.0,142.0, 141.0, 126.0, ..."
25,Height: 32 Width: 32,bird,"[0.0,0.31728875637054443, ...","[100.0, 103.0, 74.0,68.0, 91.0, 65.0, 116.0, ..."
31,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.260188102722168, ...","[127.0, 130.0, 81.0,130.0, 133.0, 88.0, ..."
33,Height: 32 Width: 32,dog,"[0.1307867169380188,0.7276672124862671, 0.0, ...","[118.0, 113.0, 81.0,122.0, 117.0, 83.0, ..."


# Exercise

## 1) Computing summary statistics of the data

Sketch summaries are techniques for computing summary statistics of data very quickly. In turicreate, SFrames and SArrays include a method:

```
.sketch_summary()
```
which computes such summary statistics. Using the training data, compute the sketch summary of the ‘label’ column and interpret the results. What’s the least common category in the training data?

In [3]:
sketch = turicreate.Sketch(image_train['label'])

In [4]:
sketch


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+------------+-------+
|   value    | count |
+------------+-------+
|    cat     |  509  |
|    dog     |  509  |
| automobile |  509  |
|    bird    |  478  |
+------------+-------+


## 2) Creating category-specific image retrieval models

In most retrieval tasks, the data we have is unlabeled, thus we call these unsupervised learning problems. However, we have labels in this image dataset, and will use these to create one model for each of the 4 image categories, {‘dog’,’cat’,’automobile’,bird’}. To start, follow these steps:

- Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the ‘label’ column equals ‘dog’, you can create an SFrame with only the data for images labeled ‘dog’.

- Similarly to the image retrieval notebook you downloaded, you are going to create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the training_data. You can call the model with the ‘dog’ data the dog_model, the one with the ‘cat’ data the cat_model, as so on.
You now have a nearest neighbors model that can find the nearest ‘dog’ to any image you give it, the dog_model; one that can find the nearest ‘cat’, the cat_model; and so on.

Using these models, answer the following questions. The cat image below is the first in the test data

You can access this image, similarly to what we did in the iPython notebooks above, with this command:

```
image_test[0:1]
```

- What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)?

### a) Split training data into 4 different categories

In [7]:
automobile = image_train.filter_by(['automobile'], 'label')
cat = image_train.filter_by(['cat'], 'label')
dog = image_train.filter_by(['dog'], 'label')
bird = image_train.filter_by(['bird'], 'label')

### b) Create nearest neighbor model for each category

In [8]:
automobile_model = turicreate.nearest_neighbors.create(automobile, features = ['deep_features'],
                                                     label = 'id')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [9]:
cat_model = turicreate.nearest_neighbors.create(cat, features = ['deep_features'],
                                                     label = 'id')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [10]:
dog_model = turicreate.nearest_neighbors.create(dog, features = ['deep_features'],
                                                     label = 'id')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [11]:
bird_model = turicreate.nearest_neighbors.create(bird, features = ['deep_features'],
                                                     label = 'id')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [12]:
image_test[0:1]

id,image,label,deep_features,image_array
0,Height: 32 Width: 32,cat,"[1.1346900463104248, 0.0,0.0, 0.0, ...","[158.0, 112.0, 49.0,159.0, 111.0, 47.0, ..."


In [13]:
image_test[0:1]['image']

dtype: Image
Rows: 1
['Height: 32 Width: 32']

In [14]:
cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 112.891ms    |

| Done         |         | 100         | 140.574ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.62371920804245,1
0,45646,36.00687992842462,2
0,32139,36.52008134363789,3
0,25713,36.754850252057054,4
0,331,36.87312281675268,5


In [15]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'], 'id')

#### What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)?

In [16]:
cat_image = image_train[image_train['id'] == 16289]
cat_image

id,image,label,deep_features,image_array
16289,Height: 32 Width: 32,cat,"[0.964287519454956, 0.0,0.0, 0.0, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."


In [17]:
get_images_from_ids(cat_model.query(image_test[0:1]))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 16.517ms     |

| Done         |         | 100         | 72.099ms     |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.5109639167785645, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
16289,Height: 32 Width: 32,cat,"[0.964287519454956, 0.0,0.0, 0.0, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."
25713,Height: 32 Width: 32,cat,"[0.5369712710380554, 0.0,0.0, 0.08944588899612 ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ..."
32139,Height: 32 Width: 32,cat,"[1.2940946817398071, 0.0,0.0, 0.5138002634048462, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ..."
45646,Height: 32 Width: 32,cat,"[0.9836775064468384, 0.0,0.0, 0.0, 0.0, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ..."


#### What is the nearest ‘dog’ labeled image in the training data to the cat image above (the first image in the test data)?

In [18]:
dog_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 10.932ms     |

| Done         |         | 100         | 78.215ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.464262878423774,1
0,13387,37.56668321685285,2
0,35867,37.60472670789396,3
0,44603,37.70655851529755,4
0,6094,38.511325490739715,5


In [19]:
dog_image = image_train[image_train['id'] == 16976]
dog_image

id,image,label,deep_features,image_array
16976,Height: 32 Width: 32,dog,"[0.7555953860282898, 0.0,0.0, 0.0, 0.0, 0.0, ...","[16.0, 17.0, 11.0, 18.0,19.0, 13.0, 20.0, 21.0, ..."


In [20]:
get_images_from_ids(dog_model.query(image_test[0:1]))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 14.057ms     |

| Done         |         | 100         | 73.134ms     |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
6094,Height: 32 Width: 32,dog,"[0.470533549785614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[91.0, 98.0, 71.0, 138.0,123.0, 63.0, 135.0, ..."
13387,Height: 32 Width: 32,dog,"[0.36649417877197266,0.0, 0.0, 0.0, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
16976,Height: 32 Width: 32,dog,"[0.7555953860282898, 0.0,0.0, 0.0, 0.0, 0.0, ...","[16.0, 17.0, 11.0, 18.0,19.0, 13.0, 20.0, 21.0, ..."
35867,Height: 32 Width: 32,dog,"[0.3053213953971863, 0.0,0.0, 0.0, 0.0, 0.0, ...","[101.0, 93.0, 9.0, 93.0,88.0, 9.0, 90.0, 86.0, ..."
44603,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 11.264665603637695, ...","[8.0, 25.0, 9.0, 29.0,39.0, 22.0, 66.0, 75.0, ..."


## 3) A simple example of nearest-neighbors classification

When we queried a nearest neighbors model, the ‘distance’ column in the table above shows the computed distance between the input and each of the retrieved neighbors. In this question, you will use these distances to perform a classification task, using the idea of a nearest-neighbors classifier.

- For the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data (similarly to what you did in the previous question). Save this result.

- Similarly, for the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’ in the training data (similarly to what you did in the previous question). Save this result.

- On average, is the first image in the test data closer to its 5 nearest neighbors in the ‘cat’ data or in the ‘dog’ data? (In a later course, we will see that this is an example of what is called a k-nearest neighbors classifier, where we use the label of neighboring points to predict the label of a test point.)

In [21]:
cat_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 9.516ms      |

| Done         |         | 100         | 83.64ms      |

+--------------+---------+-------------+--------------+

36.15573070978294

In [22]:
dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 10.629ms     |

| Done         |         | 100         | 84.696ms     |

+--------------+---------+-------------+--------------+

37.77071136184156

## 4) [Challenging Question] Computing nearest neighbors accuracy using SFrame operations

A nearest neighbor classifier predicts the label of a point as the most common label of its nearest neighbors. In this question, we will measure the accuracy of a 1-nearest-neighbor classifier, i.e., predict the output as the label of the nearest neighbor in the training data. Although there are simpler ways of computing this result, we will go step-by-step here to introduce you to more concepts in nearest neighbors and SFrames, which will be useful later in this Specialization.

- Training models: For this question, you will need the nearest neighbors models you learned above on the training data, i.e., the dog_model, cat_model, automobile_model and bird_model.

- Spliting test data by label: Above, you split the train data SFrame into one SFrame for images labeled ‘dog’, another for those labeled ‘cat’, etc. Now, do the same for the test data. You can call the resulting SFrames

```
image_test_cat, image_test_dog, image_test_bird, image_test_automobile
```

In [23]:
image_test_automobile = image_test.filter_by(['automobile'], 'label')
image_test_cat = image_test.filter_by(['cat'], 'label')
image_test_dog = image_test.filter_by(['dog'], 'label')
image_test_bird = image_test.filter_by(['bird'], 'label')

In [25]:
image_test_cat.head(6)

id,image,label,deep_features,image_array
0,Height: 32 Width: 32,cat,"[1.1346900463104248, 0.0,0.0, 0.0, ...","[158.0, 112.0, 49.0,159.0, 111.0, 47.0, ..."
8,Height: 32 Width: 32,cat,"[0.0, 0.0,0.034419238567352295, ...","[23.0, 19.0, 23.0, 19.0,21.0, 28.0, 21.0, 16.0, ..."
46,Height: 32 Width: 32,cat,"[0.0, 0.0,0.25575894117355347, ...","[22.0, 27.0, 19.0, 30.0,36.0, 26.0, 30.0, 36.0, ..."
53,Height: 32 Width: 32,cat,"[0.6107316613197327, 0.0,0.0, 0.0, 0.0, 0.0, ...","[255.0, 255.0, 251.0,249.0, 251.0, 246.0, ..."
61,Height: 32 Width: 32,cat,"[0.9359390735626221, 0.0,0.7783024311065674, 0.0, ...","[10.0, 10.0, 10.0, 10.0,10.0, 10.0, 10.0, 10.0, ..."
63,Height: 32 Width: 32,cat,"[0.0, 0.0,0.3594752550125122, 0.0, ...","[233.0, 225.0, 209.0,201.0, 170.0, 123.0, ..."


- Finding nearest neighbors in the training set for each part of the test set: Thus far, we have queried, e.g.,

```
dog_model.query()
```

our nearest neighbors models with a single image as the input, but you can actually query with a whole set of data, and it will find the nearest neighbors for each data point. Note that the input index will be stored in the ‘query_label’ column of the output SFrame.

Using this knowledge find the closest neighbor in to the dog test data using each of the trained models, e.g.,

```
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
```

finds 1 neighbor (that’s what k=1 does) to the dog test images (image_test_dog) in the cat portion of the training data (used to train the cat_model).

Now, do this for every combination of the labels in the training and test data.

- Create an SFrame with the distances from ‘dog’ test examples to the respective nearest neighbors in each class in the training data: The ‘distance’ column in dog_cat_neighbors above contains the distance between each ‘dog’ image in the test set and its nearest ‘cat’ image in the training set. The question we want to answer is how many of the test set ‘dog’ images are closer to a ‘dog’ in the training set than to a ‘cat’, ‘automobile’ or ‘bird’. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called dog_distances with 4 columns:

i. dog_distances[‘dog-dog’] ---- storing dog_dog_neighbors[‘distance’]

ii. dog_distances[‘dog-cat’] ---- storing dog_cat_neighbors[‘distance’]

iii. dog_distances[‘dog-automobile’] ---- storing dog_automobile_neighbors[‘distance’]

iv. dog_distances[‘dog-bird’] ---- storing dog_bird_neighbors[‘distance’]

In [26]:
dog_cat_neighbors = cat_model.query(image_test_dog, k = 1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 532.249ms    |

| Done         | 509000  | 100         | 594.071ms    |

+--------------+---------+-------------+--------------+

In [27]:
dog_dog_neighbors = dog_model.query(image_test_dog, k = 1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 500.746ms    |

| Done         | 509000  | 100         | 594.033ms    |

+--------------+---------+-------------+--------------+

In [28]:
dog_automobile_neighbors = automobile_model.query(image_test_dog, k = 1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 544.831ms    |

| Done         | 509000  | 100         | 572.453ms    |

+--------------+---------+-------------+--------------+

In [29]:
dog_bird_neighbors = bird_model.query(image_test_dog, k = 1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 119000  | 24.8954     | 556.387ms    |

| Done         | 478000  | 100         | 679.842ms    |

+--------------+---------+-------------+--------------+

Hint: You can create a new SFrame from the columns of other SFrames by creating a dictionary with the new columns, as shown in this example:

```
new_sframe = turicreate.SFrame({‘foo’: other_sframe[‘foo’],‘bar’: some_other_sframe[‘bar’]})
```


In [30]:
dog_distances = turicreate.SFrame({'dog_automobile': dog_automobile_neighbors['distance'],
                                   'dog_bird': dog_bird_neighbors['distance'],
                                   'dog_cat': dog_cat_neighbors['distance'],
                                   'dog_dog': dog_dog_neighbors['distance']
                                 })

In [31]:
dog_distances.head()

dog_automobile,dog_bird,dog_cat,dog_dog
41.95797614571203,41.75386473035126,36.419607706754384,33.47735903726335
46.00213318067788,41.3382958924861,38.83532688735542,32.84584956840554
42.946229069238804,38.615759085289056,36.97634108541546,35.03970731890584
41.68660600484793,37.08922699538214,34.575007291446106,33.90103276968193
39.22696649347584,38.27228869398105,34.77882479101661,37.484925090925636
40.58451176980721,39.146208923590486,35.11715782924591,34.94516534398124
45.10673529610854,40.523040105962316,40.60958309132649,39.095727834463545
41.32211409739762,38.19479183926956,39.90368673062214,37.76961310322034
41.82446549950164,40.156713166131446,38.067470016821176,35.10891446032838
45.497692940110376,45.55979626027668,42.72587329506032,43.242283258453455


- Computing the number of correct predictions using 1-nearest neighbors for the dog class: Now that you have created the SFrame dog_distances, you will learn to use the method

```
.apply()
```

on this SFrame to iterate line by line and compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes. You will do this in three steps:

i. Consider one row of the SFrame dog_distances. Let’s call this variable row. You can access each distance by calling, for example,

```
row[‘dog_cat’]
```

which, in example table above, will have value equal to 36.4196077068 for the first row.

Create a function starting with

```
def is_dog_correct(row):
```

which returns 1 if the value for row[‘dog_dog’] is lower than that of the other columns, and 0 otherwise. That is, returns 1 if this row is correctly classified by 1-nearest neighbors, and 0 otherwise.

In [32]:
def is_dog_correct(row):  
    if row['dog_dog'] <= min(row.values()):     
        return 1    
    else:        
        return 0

# dog_distances.apply(lambda row: 1 if row['dog_dog'] <= min(row.values()) else 0)    

ii. Using the function is_dog_correct(row), you can check if 1 row is correctly classified. Now, you want to count how many rows are correctly classified. You could do a for loop iterating through each row and applying the function is_dog_correct(row). This method will be really slow, because the SFrame is not optimized for this type of operation.

Instead, we will use the .apply() method to iterate the function is_dog_correct for each row of the SFrame. 

[Read about using the .apply() method here](https://eventing.coursera.org/api/redirectStrict/zssXfP7-T1FOiLHfvCOcezVm_aoU_xR7M6Edy661mCWCb7Zm9bcu05SUehwDyR_58HoYuVu4bhqUmVhY4N8HNQ.iPRoCqpTXPkd10lpgN9yUQ.pFPbAJXAMJhxS2yCKuTEdVA5xtWRnaDSooDTRZRXedc3pMvHBrqGCNdrmYqTDorFXK0lTmM7ZQFHdRrzXlZB8chyucFzFlACThHONWAbCUxC1pfORR8uLMu5hiwNuiAlDwHH8CHUf0XdFzgNU3aJlMTBVXwEi3Qniv6Z_KJsIDJVvHL4OR5bViQwLYyXlimhzs_xioqOgsPtLg3_bcF47Yl3l2aJ8HSuORlTkyqe0tlRQERlfv6XV4sexvCgKFdvwtYBkF5bfddusF7wSLoETbWuAQ4mOLt35SgHq-4WUF8hrs8_PnzWOGo-v1Wu7toFSFuzacIZdsOUGuX8ocHSirQ-vaxC9fHF4QtwrG4L_60).

iii. Computing the number of correct predictions for ‘dog’: You can now call:
        
```
dog_distances.apply(is_dog_correct)
```
which will return an SArray (a column of data) with a 1 for every correct row and a 0 for every incorrect one. 

In [33]:
dog_distances.apply(is_dog_correct)

dtype: int
Rows: 1000
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ... ]

You can call:

```
.sum()
```
on the result to get the total number of correctly classified ‘dog’ images in the test set!

In [34]:
dog_distances.apply(is_dog_correct).sum()

678

Hint: To make sure your code is working correctly, if you were to do steps d) and e) in this question to count the number of correctly classified ‘cat’ images in the test data, instead of ‘dog’, the result would be 548.

In [35]:
cat_distances = turicreate.SFrame({'cat_automobile': automobile_model.query(image_test_cat, k = 1)['distance'],
                                   'cat_bird': bird_model.query(image_test_cat, k = 1)['distance'],
                                   'cat_cat': cat_model.query(image_test_cat, k = 1)['distance'],
                                   'cat_dog': dog_model.query(image_test_cat, k = 1)['distance'],
                                 })

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 495.567ms    |

| Done         | 509000  | 100         | 563.622ms    |

+--------------+---------+-------------+--------------+

In [36]:
cat_distances.head()

cat_automobile,cat_bird,cat_cat,cat_dog
39.67105827921273,38.07426586900459,34.62371920804243,37.464262878423845
43.00890566881215,36.36740241377261,33.868057930239424,29.347231958505894
38.60100066036172,35.303939494706285,32.46151689024062,32.259964047497306
39.35663070911102,38.89440296011285,35.77082102542339,35.385208518819475
38.3572372618227,34.28204098747585,31.157768641742653,30.04429850876174
42.09047931808273,44.53521701779428,41.398603584704766,35.4741000423819
39.05202512531326,34.0290595084087,30.989459495892984,32.58452752259978
39.30586450690295,39.02369249827745,37.08146073866276,37.650285261439834
43.02481297990393,40.83340542972473,39.988386368829005,36.98013535120841
45.67491764256238,40.12588356010239,39.70766330973091,41.12594107067673


In [45]:
cat_distances['cat_cat'].min()

18.59484029811513

In [46]:
cat_distances['cat_cat'].max()

51.92207833867718

In [39]:
cat_distances.apply(lambda row: 1 if row['cat_cat'] <= min(row.values()) else 0)    

dtype: int
Rows: 1000
[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, ... ]

In [40]:
cat_distances.apply(is_cat_correct).sum()

548

- Accuracy of predicting dog in the test data: Using the work you did in this question, what is the accuracy of the 1-nearest neighbor classifier at classifying ‘dog’ images from the test set?

In [41]:
dog_distances.apply(is_dog_correct).sum() / float(len(dog_distances))

0.678